In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Downloading a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-06-29 04:17:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.63MB/s    in 0.2s    

2020-06-29 04:17:52 (3.63 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Starting a Spark session with an additional option that adds the driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon_Reviews_analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Connectioning to the DB and pulling the data fron the vine_table
jdbc_url="jdbc:postgresql://dataviz.czruykdkti9i.us-east-2.rds.amazonaws.com:5432/amazon_reviews"
config = {"user":"postgres", 
          "password": "___________________", 
          "driver":"org.postgresql.Driver"}
my_df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties= config)
my_df.limit(10).show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+



In the table "vine_table" there is no column with verified/unverified purchase data so I prefer to analyse the initial source from S3 because I would like to include it in my analysis.

In [ ]:
from pyspark.sql.types import StructField,IntegerType, StructType,StringType, DateType

In [ ]:
# Defining data type for each column according to the data type in the RDS (before extracting data set from S3)
newDF=[StructField('marketplace',StringType(),True),
       StructField('customer_id',IntegerType(),True),
       StructField('review_id',StringType(),True),
       StructField('product_id',StringType(),True),
       StructField('product_parent',IntegerType(),True),
       StructField('product_title',StringType(),True),
       StructField('product_category',StringType(),True),
       StructField('star_rating',IntegerType(),True),
       StructField('helpful_votes',IntegerType(),True),
       StructField('total_votes',IntegerType(),True),
       StructField('vine',StringType(),True),
       StructField('verified_purchase',StringType(),True),
       StructField('review_headline',StringType(),True),
       StructField('review_body',StringType(),True),
       StructField('review_date',DateType(),True)
       ]
finalStruct=StructType(fields=newDF)

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, schema=finalStruct)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
df.schema

StructType(List(StructField(marketplace,StringType,true),StructField(customer_id,IntegerType,true),StructField(review_id,StringType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(product_title,StringType,true),StructField(product_category,StringType,true),StructField(star_rating,IntegerType,true),StructField(helpful_votes,IntegerType,true),StructField(total_votes,IntegerType,true),StructField(vine,StringType,true),StructField(verified_purchase,StringType,true),StructField(review_headline,StringType,true),StructField(review_body,StringType,true),StructField(review_date,DateType,true)))

In [ ]:
# Creating a dataframe for analysis - dropping columns that will not be used
analysis_df = df.select(["review_id","product_id","product_parent","star_rating","helpful_votes","total_votes","vine","verified_purchase","review_date"]) 
analysis_df.show()

+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|product_id|product_parent|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| RTIS3L2M1F5SM|B001CXYMFS|     737716809|          5|            0|          0|   N|                Y| 2015-08-31|
| R1ZV7R40OLHKD|B00M920ND6|     569686175|          5|            0|          0|   N|                Y| 2015-08-31|
|R3BH071QLH8QMC|B0029CSOD2|      98937668|          1|            0|          1|   N|                Y| 2015-08-31|
|R127K9NTSXA2YH|B00GOOSV98|      23143350|          3|            0|          0|   N|                Y| 2015-08-31|
|R32ZWUXDJPW27Q|B00Y074JOM|     821342511|          4|            0|          0|   N|                Y| 2015-08-31|
|R3AQQ4YUKJWBA6|B002UBI6W6|     328764615|          1|            0|    

In [ ]:
analysis_df.describe("star_rating","helpful_votes","total_votes").show()

+-------+------------------+------------------+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|
+-------+------------------+------------------+-----------------+
|  count|           1785997|           1785997|          1785997|
|   mean| 4.059892597803915|2.2593492598251843|3.757368573407458|
| stddev|1.3558117532896532| 19.17128509230589|22.45439170150866|
|    min|                 1|                 0|                0|
|    max|                 5|             10498|            10780|
+-------+------------------+------------------+-----------------+



Fron this table we get a brief ststistical description. Let's break it down to vine / non-vine data and drill down to verified / unverified perchase.

In [ ]:
vine_df = analysis_df.filter(analysis_df["vine"] == "Y")
vine_df.show()

+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|product_id|product_parent|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| RX3CK2RLSAF7T|B00XO041RQ|     238654494|          5|            0|          0|   Y|                N| 2015-08-31|
|R253CC14DRG5WQ|B00XO041RQ|     238654494|          3|            0|          0|   Y|                N| 2015-08-31|
| R1ZXWP3JKVXMU|B00XO041RQ|     238654494|          5|            0|          0|   Y|                N| 2015-08-30|
|R1I01YYQ35M80G|B00YY3OK0A|     884170609|          5|            0|          0|   Y|                N| 2015-08-30|
| RWKQQPG100CQ4|B00YY3ODTS|     975991516|          4|            2|          4|   Y|                N| 2015-08-28|
|R1R0J8S1VGALUY|B00UOYQ5DS|     166248134|          1|            1|    

In [ ]:
desc_vine = vine_df.describe("star_rating","helpful_votes","total_votes")
desc_vine.show()

+-------+------------------+-----------------+------------------+
|summary|       star_rating|    helpful_votes|       total_votes|
+-------+------------------+-----------------+------------------+
|  count|              4291|             4291|              4291|
|   mean| 4.074807737124213|2.348403635516197|3.2780237706828244|
| stddev|0.9182159041910316|12.57900104398382|13.768374606698877|
|    min|                 1|                0|                 0|
|    max|                 5|              347|               362|
+-------+------------------+-----------------+------------------+



In [ ]:
verif_vine_df = vine_df.filter(vine_df["verified_purchase"] == "Y")
verif_vine_df.show()

+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|product_id|product_parent|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|R2IORRLBT74A3Z|B00KAED850|     602445240|          4|            0|          0|   Y|                Y| 2014-12-30|
|R24XJUUZJHAL9J|B006IOAHTQ|     894175424|          5|            1|          1|   Y|                Y| 2014-04-13|
|R22L2W44Q5A126|B00AXI9WFS|     237701427|          5|            0|          0|   Y|                Y| 2014-02-20|
|R28UQK039G2IYF|B00EZKNT18|     838287039|          5|            0|          0|   Y|                Y| 2014-01-29|
|R2JPAYSJQ3UBL0|B0094MYU4M|     701734383|          5|            0|          0|   Y|                Y| 2014-01-08|
|R21776NRZUUN49|B00ER8HHDO|     710428259|          4|            1|    

In [ ]:
desc_vine_verif = verif_vine_df.describe("star_rating","helpful_votes","total_votes")
desc_vine_verif.show()

+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|                22|                22|                22|
|   mean| 4.454545454545454|1.8636363636363635|3.1818181818181817|
| stddev|0.9625003513801572| 3.196656803382984| 5.029781436474283|
|    min|                 1|                 0|                 0|
|    max|                 5|                12|                15|
+-------+------------------+------------------+------------------+



In [ ]:
non_verif_vine_df = vine_df.filter(vine_df["verified_purchase"] == "N")
non_verif_vine_df.show()

+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|product_id|product_parent|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| RX3CK2RLSAF7T|B00XO041RQ|     238654494|          5|            0|          0|   Y|                N| 2015-08-31|
|R253CC14DRG5WQ|B00XO041RQ|     238654494|          3|            0|          0|   Y|                N| 2015-08-31|
| R1ZXWP3JKVXMU|B00XO041RQ|     238654494|          5|            0|          0|   Y|                N| 2015-08-30|
|R1I01YYQ35M80G|B00YY3OK0A|     884170609|          5|            0|          0|   Y|                N| 2015-08-30|
| RWKQQPG100CQ4|B00YY3ODTS|     975991516|          4|            2|          4|   Y|                N| 2015-08-28|
|R1R0J8S1VGALUY|B00UOYQ5DS|     166248134|          1|            1|    

In [ ]:
desc_vine_unverif = non_verif_vine_df.describe("star_rating","helpful_votes","total_votes")
desc_vine_unverif.show()

+-------+------------------+------------------+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|
+-------+------------------+------------------+-----------------+
|  count|              4269|              4269|             4269|
|   mean| 4.072850784727103|  2.35090185055048|3.278519559615835|
| stddev|0.9176933368168877|12.609337640900852| 13.7993031913543|
|    min|                 1|                 0|                0|
|    max|                 5|               347|              362|
+-------+------------------+------------------+-----------------+



In [ ]:
non_vine_df = analysis_df.filter(analysis_df["vine"] == "N")
non_vine_df.show(5)

+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|product_id|product_parent|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| RTIS3L2M1F5SM|B001CXYMFS|     737716809|          5|            0|          0|   N|                Y| 2015-08-31|
| R1ZV7R40OLHKD|B00M920ND6|     569686175|          5|            0|          0|   N|                Y| 2015-08-31|
|R3BH071QLH8QMC|B0029CSOD2|      98937668|          1|            0|          1|   N|                Y| 2015-08-31|
|R127K9NTSXA2YH|B00GOOSV98|      23143350|          3|            0|          0|   N|                Y| 2015-08-31|
|R32ZWUXDJPW27Q|B00Y074JOM|     821342511|          4|            0|          0|   N|                Y| 2015-08-31|
+--------------+----------+--------------+-----------+-------------+----

In [ ]:
desc_non_vine = non_vine_df.describe("star_rating","helpful_votes","total_votes")
desc_non_vine.show()

+-------+------------------+-----------------+------------------+
|summary|       star_rating|    helpful_votes|       total_votes|
+-------+------------------+-----------------+------------------+
|  count|           1781706|          1781706|           1781706|
|   mean| 4.059856676690767|2.259134784302236| 3.758523011091617|
| stddev|1.3566952586970766| 19.1844293543374|22.471248446467634|
|    min|                 1|                0|                 0|
|    max|                 5|            10498|             10780|
+-------+------------------+-----------------+------------------+



In [ ]:
verif_non_vine_df = non_vine_df.filter(non_vine_df["verified_purchase"] == "Y")
verif_non_vine_df.show(5)

+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|product_id|product_parent|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| RTIS3L2M1F5SM|B001CXYMFS|     737716809|          5|            0|          0|   N|                Y| 2015-08-31|
| R1ZV7R40OLHKD|B00M920ND6|     569686175|          5|            0|          0|   N|                Y| 2015-08-31|
|R3BH071QLH8QMC|B0029CSOD2|      98937668|          1|            0|          1|   N|                Y| 2015-08-31|
|R127K9NTSXA2YH|B00GOOSV98|      23143350|          3|            0|          0|   N|                Y| 2015-08-31|
|R32ZWUXDJPW27Q|B00Y074JOM|     821342511|          4|            0|          0|   N|                Y| 2015-08-31|
+--------------+----------+--------------+-----------+-------------+----

In [ ]:
desc_non_vine_verif = verif_non_vine_df.describe("star_rating","helpful_votes","total_votes")
desc_non_vine_verif.show()

+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|           1169679|           1169679|           1169679|
|   mean| 4.191520921551981|1.0210562043090454|1.7537367089603215|
| stddev|1.2794620222361162|10.399193366336865|12.093767292486723|
|    min|                 1|                 0|                 0|
|    max|                 5|              3744|              4066|
+-------+------------------+------------------+------------------+



In [ ]:
non_verif_non_vine_df = non_vine_df.filter(non_vine_df["verified_purchase"] == "N")
non_verif_non_vine_df.show(5)

+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|product_id|product_parent|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+--------------+----------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|R1VTIA3JTYBY02|B00008KTNN|     384411423|          5|            0|          0|   N|                N| 2015-08-31|
|R23HC6NMA3HBPS|B00MV7KVP4|      92208606|          5|            1|          1|   N|                N| 2015-08-31|
|R2MTHHQM6RSDQK|B00503E8S2|     895635946|          4|            0|          1|   N|                N| 2015-08-31|
| R4PKAZRQJJX14|B00QZLVCU0|     210935604|          1|           21|         34|   N|                N| 2015-08-31|
|R2BIY6KUTEBIJB|B00NY5ZPF8|     820511031|          5|            0|          3|   N|                N| 2015-08-31|
+--------------+----------+--------------+-----------+-------------+----

In [ ]:
desc_non_vine_unverif = non_verif_non_vine_df.describe("star_rating","helpful_votes","total_votes")
desc_non_vine_unverif.show()

+-------+------------------+-----------------+------------------+
|summary|       star_rating|    helpful_votes|       total_votes|
+-------+------------------+-----------------+------------------+
|  count|            612027|           612027|            612027|
|   mean|3.8082257808887516|4.625295942826051|7.5899821413107595|
| stddev| 1.460577154870917|29.26128497742793| 34.17783495286794|
|    min|                 1|                0|                 0|
|    max|                 5|            10498|             10780|
+-------+------------------+-----------------+------------------+



In [ ]:
# Comparing total vine and non-vine reviews
summary_join = desc_vine.join(desc_non_vine, on=["summary"], how="inner")
summary_join.show()

+-------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|summary|       star_rating|    helpful_votes|       total_votes|       star_rating|    helpful_votes|       total_votes|
+-------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|  count|              4291|             4291|              4291|           1781706|          1781706|           1781706|
|   mean| 4.074807737124213|2.348403635516197|3.2780237706828244| 4.059856676690767|2.259134784302236| 3.758523011091617|
| stddev|0.9182159041910316|12.57900104398382|13.768374606698877|1.3566952586970766| 19.1844293543374|22.471248446467634|
|    min|                 1|                0|                 0|                 1|                0|                 0|
|    max|                 5|              347|               362|                 5|            10498|             10780|
+-------+---------------

The analysis of total reviews that we compare in this table shows us the difference in standard deviation - the non-vine data is more spread from the mean. That means that non-vine data has more negative reviews than vine so vine could be biased. Moreover, the number of votes in vine reviews is higher compared to the non-vine - 8% vs 0.6% respective. But the number of reviews also matters - the number of vine reviews is too small (0.2% from total number of reviews) so there is a possibility that results can be different if we collect more data. 

In [ ]:
# Comparing verified vine and non-vine reviews
summary_join_verif = desc_vine_verif.join(desc_non_vine_verif, on=["summary"], how="inner")
summary_join_verif.show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|                22|                22|                22|           1169679|           1169679|           1169679|
|   mean| 4.454545454545454|1.8636363636363635|3.1818181818181817| 4.191520921551981|1.0210562043090454|1.7537367089603215|
| stddev|0.9625003513801572| 3.196656803382984| 5.029781436474283|1.2794620222361162|10.399193366336865|12.093767292486723|
|    min|                 1|                 0|                 0|                 1|                 0|                 0|
|    max|                 5|                12|                15|                 5|              3744|              4066|
+-------

If we take a look at verified purchases and compare vine and non-vine, the outcome is not too different from the result above.

In [ ]:
# Comparing unverified vine and non-vine reviews
summary_join_unverif = desc_vine_unverif.join(desc_non_vine_unverif, on=["summary"], how="inner")
summary_join_unverif.show()

+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|summary|       star_rating|     helpful_votes|      total_votes|       star_rating|    helpful_votes|       total_votes|
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|  count|              4269|              4269|             4269|            612027|           612027|            612027|
|   mean| 4.072850784727103|  2.35090185055048|3.278519559615835|3.8082257808887516|4.625295942826051|7.5899821413107595|
| stddev|0.9176933368168877|12.609337640900852| 13.7993031913543| 1.460577154870917|29.26128497742793| 34.17783495286794|
|    min|                 1|                 0|                0|                 1|                0|                 0|
|    max|                 5|               347|              362|                 5|            10498|             10780|
+-------+---------------

But the data of unverified purchases tells that there is big difference between vine and non-vine reviews - the mean of unverified is much lower and the data is spread from the mean significantly. There is more reasurch and investigation needed on this because it can be caused by competitors writing negative reviews.
We need to collect more data for the vine reviews analysis as well to make a conclusion is the reviews are biased.